http://www.menpo.org/menpofit/atm.html

In [ ]:
from pathlib import Path
import menpo.io as mio

path_to_lfpw = Path('/home/rshaydu/tensorfaces/lfpw/trainset/')

image = mio.import_image(path_to_lfpw / 'image_0002.png')
image = image.crop_to_landmarks_proportion(0.5)

template = mio.import_image(path_to_lfpw / 'image_0005.png')
template = template.crop_to_landmarks_proportion(0.5)


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.subplot(121)
image.view()
plt.gca().set_title('Input Image')

plt.subplot(122)
template.view_landmarks(marker_face_colour='white', marker_edge_colour='black',
                        marker_size=4)
plt.gca().set_title('Template');


In [ ]:
from menpo.visualize import print_progress

training_shapes = []
for lg in print_progress(mio.import_landmark_files(path_to_lfpw / '*.pts', verbose=True)):
    training_shapes.append(lg)


In [ ]:
from menpofit.atm import HolisticATM
from menpo.feature import igo
from menpofit.transform import DifferentiableThinPlateSplines

atm = HolisticATM(template, training_shapes, group='PTS',
                  diagonal=180, scales=(0.25, 1.0),
                  holistic_features=igo, verbose=True, transform=DifferentiableThinPlateSplines)


In [ ]:
template.view()

In [ ]:
atm.view_shape_models_widget()



In [ ]:
atm.view_atm_widget()

In [ ]:
from menpofit.atm import LucasKanadeATMFitter, InverseCompositional

fitter = LucasKanadeATMFitter(atm,
                              lk_algorithm_cls=InverseCompositional, n_shape=[5, 15])


In [ ]:
print(fitter)

In [ ]:
from menpodetect import load_dlib_frontal_face_detector

# Load detector
detect = load_dlib_frontal_face_detector()

image = mio.import_image('/home/rshaydu/tensorfaces/FaceBase_png/amir-vp0-il1-ex2.png')

# Detect
bboxes = detect(image)
print("{} detected faces.".format(len(bboxes)))

# View
if len(bboxes) > 0:
    image.view_landmarks(group='dlib_0', line_colour='white',
                         render_markers=False, line_width=3);


In [ ]:
# initial bbox
initial_bbox = bboxes[0]

# fit image
result = fitter.fit_from_bb(image, initial_bbox, max_iters=200)

# print result
print(result)


In [ ]:
result.view(render_initial_shape=True)
result.image.shape

In [ ]:
from menpofit.transform import DifferentiablePiecewiseAffine

warped = fitter.warped_images(result.image, [result.final_shape, fitter.reference_shape])
warped[0].shape